# 用 Python 做量化投资

## 注册免费的量化投资账号

国内有不少量化投资网站，比如 https://uqer.datayes.com/

可以注册免费的账号，新建策略。

下面是一个策略的例子：

In [ ]:
import pandas as pd
start = '2014-01-01'                       # 回测起始时间
end = '2018-01-01'                         # 回测结束时间
benchmark = 'HS300'                        # 策略参考标准
universe = DynamicUniverse('HS300')  # 股票池为沪深300
freq = 'd'                                 # 策略类型，'d'表示日间策略使用日线回测，'m'表示日内策略使用分钟线回测
refresh_rate = 60                           # 调仓频率设为60天，表示执行handle_data的时间间隔，若freq = 'd'时间间隔的单位为交易日，若freq = 'm'时间间隔为分钟
max_history_window = 60
  
# 配置账户信息，支持多资产多账户
accounts = {
    'fantasy_account': AccountConfig(account_type='security', capital_base=10000000)
}
  
def initialize(context):
    pass
  
# 每个单位时间(如果按天回测,则每天调用一次,如果按分钟,则每分钟调用一次)调用一次
def handle_data(context):    
    account = context.get_account('fantasy_account')   
    universe = context.get_universe(exclude_halt=True)
    history = context.history(universe, 'closePrice', 60)
    momentum = {'symbol':[], 'c_ret':[]}
    for stk in history.keys():
        momentum['symbol'].append(stk)
        momentum['c_ret'].append(history[stk]['closePrice'][-1]/history[stk]['closePrice'][0])
    # 按照过去60个交易日的收益率排序，并且选择前60只股票作为买入候选
    momentum = pd.DataFrame(momentum).sort(columns = 'c_ret', ascending = False).reset_index()
    # print momentum
    momentum = momentum[:60]
    buylist = momentum['symbol'].tolist()
    
    for stk in account.get_positions():
        if stk not in buylist:
            account.order_to(stk, 0)
    
    # 等权买入所选股票
    portfolio_value = account.portfolio_value
    for stk in buylist:
        account.order_pct_to(stk, 1.0/len(buylist))